In [267]:
import yaml
import json
import requests
import pandas as pd

from PIL import Image
from io import BytesIO
from bs4 import BeautifulSoup

In [200]:
# main url
with open('config/news_homepage.yaml', 'r') as f:
    main_url = yaml.load(f, Loader=yaml.FullLoader)

# category
category = pd.read_excel('data/news_category_eng.xlsx', index_col=0, sheet_name=None)
main_category = list(category.keys())
# main_category
main_url

{'chosun': 'https://www.chosun.com/',
 'joongang': 'https://www.joongang.co.kr/',
 'donga': 'https://www.donga.com/news/',
 'seoul': 'https://www.seoul.co.kr/news/newsList.php?section=politics',
 'khan': 'https://www.khan.co.kr/',
 'maeil': 'https://news.imaeil.com/',
 'hani': 'https://www.hani.co.kr/arti/',
 'hankook': 'https://hankookilbo.com/News'}

In [255]:
# chosun / sports
news_comp = 'joongang'
news_list_url = main_url[news_comp] + f"{main_category[-1]}/" + f"{category['sports'][news_comp].loc['category2-1']}/"

news_list_url = "https://www.joongang.co.kr/sports/soccer?page=1"
news_list_url

'https://www.joongang.co.kr/sports/soccer?page=1'

In [253]:
res = requests.get(news_list_url)
html = BeautifulSoup(res.text, 'html.parser')

## 뉴스리스트에서 상세페이지 url 추출

In [258]:
news_list = html.find_all('h2', 'headline')
news_detail_urls = [item.find('a').attrs['href'] for item in news_list]
news_id = [urls.split("/")[-1] for urls in news_detail_urls]

['25180796',
 '25180792',
 '25180784',
 '25180768',
 '25180766',
 '25180760',
 '25180759',
 '25180741',
 '25180737',
 '25180713',
 '25180672',
 '25180440',
 '25180414',
 '25180387',
 '25180342',
 '25180328',
 '25180271',
 '25180267',
 '25180118',
 '25180116',
 '25180078',
 '25180010',
 '25180003',
 '25179970']

## 각 상세페이지로 접속하여 제목, 본문, 기사, 시간, 이미지 추출

In [278]:
for url in news_detail_urls:
    res = requests.get(url)
    html = BeautifulSoup(res.text, 'html.parser')
    
    news_titles = html.find_all('h1', 'headline')
    news_titles = list(map(lambda x: x.text.replace('\n', '').replace('\'', '').strip(), news_titles))
    
    news_contents = html.find_all('div', 'article_body fs3')
    news_contents = list(map(lambda x: x.text.replace('\n', '').replace('\'', '').strip(), news_contents))

    news_author = []
    author = html.find('div', 'byline').find('a')
    author = author.text.split()[0]
    news_author.append(author)

    news_insert_dt = []
    news_update_dt = []
    
    try:
        insert_dt = html.find('p', 'date').find('time').attrs['datetime']
        news_insert_dt.append(insert_dt)
    except:
        news_insert_dt.append('None')

    try:
        update_dt = html.find('p', 'update').find('time').attrs['datetime']
    except:
        news_update_dt.append('None')

    news_detail_images = html.find_all('div', 'ab_photo')
    news_detail_images = [images.find('img').attrs['data-src'] for images in news_detail_images]    

## 이미지 저장

In [275]:
for image in news_detail_images:
    response = requests.get(image)
    img = Image.open(BytesIO(response.content))
    # img.show() # 이미지 확인
    img.save(f'/Users/jaeb/news/static/news_images/joongang/{news_id}/{image.split("/")[-1]}')

In [229]:
news_titles, news_contents, news_author, news_insert_dt

(['올 시즌 셀틱에선 한국인 3명 뛰는 진풍경 본다...권혁규 영입 발표'],
 ['셀틱의 세 번째 한국인 선수가 된 권혁규. 사진 셀틱 SNS        \xa0올 시즌 유럽 프로축구에는 한국 선수 세 명이 동시에 그라운드를 누비는 진풍경을 볼 수 있다. 스코틀랜드 명문 셀틱FC에서다.   셀틱 구단은 24일 "권혁규(22)와 5년 계약을 맺었다"고 밝혔다. 등번호는 22번을 배정받았다. 권혁규는\xa0소속팀 부산 아이파크(K리그2)가 지난 18일 그의 셀틱행을 알리면서 이적이 알려졌다. 이적료는 100만 유로(약 14억원). 셀틱은 2022~23시즌 스코틀랜드 무대에서 정규리그, 스코티시컵(FA컵), 리그컵을 석권하며 통산 8번째 트레블(3관왕)을 이뤘다. 잉글랜드 프리미어리그(EPL)에 참가한다면 하위권 정도의 전력을 갖춘 것으로 평가 받는다.   이로써 셀틱은 지난 1월 수원 삼성(K리그 1부)에서 영입한 공격수 오현규(22), 지난 15일 셀틱 이적을 발표한 강원FC 공격수 양현준(21)에 이어 권혁규까지 영입하며 한국인 선수만 세 명을 보유한 구단이 됐다. 셀틱은 과거 미드필더 기성용(34·FC서울), 수비수 차두리(43·한국 축구대표팀 코디네이터)가 몸담았던 팀으로 한국 팬들에게도 익숙하다.   셀틱은 지난해 12월 한 차례 권혁규에게 러브콜을 보내는 등 오래 전부터 그를 관찰했다. 수비형 미드필더 권혁규는 개성고 3학년이던 2019년 부산에서 프로 데뷔했다. K리그 통산 76경기 3골 2도움을 기록했다. 키 1m90㎝, 77㎏의 탄탄한 체격에 발 기술까지 뛰어나다는 평가를 받는다. 2021~22시즌 군팀 김천 상무에서 뛰며 병역 문제를 해결한 것도 이번 유럽행에 유리하게 작용했다.   셀틱은 최근 몇 년간 일본 J리그 출신 선수들을 여러 명 영입해 좋은 성과를 냈다. 올해는 한국 선수들 영입으로 이어졌다. 현재 셀틱 로스터에 오른 한국·일본 선수는 오현규와 일본의 마에다 다이젠, 후루하시 교고, 이와타 도모키, 하타테 레오, 고바야시 유키 등